<div align="center">

  <a href="https://ultralytics.com/yolov8" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>


<br>
  <a href="https://console.paperspace.com/github/ultralytics/ultralytics"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"/></a>
  <a href="https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://www.kaggle.com/ultralytics/yolov8"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
<br>

Welcome to the Ultralytics YOLOv8 🚀 notebook! <a href="https://github.com/ultralytics/ultralytics">YOLOv8</a> is the latest version of the YOLO (You Only Look Once) AI models developed by <a href="https://ultralytics.com">Ultralytics</a>. This notebook serves as the starting point for exploring the various resources available to help you get started with YOLOv8 and understand its features and capabilities.

YOLOv8 models are fast, accurate, and easy to use, making them ideal for various object detection and image segmentation tasks. They can be trained on large datasets and run on diverse hardware platforms, from CPUs to GPUs.

We hope that the resources in this notebook will help you get the most out of YOLOv8. Please browse the YOLOv8 <a href="https://docs.ultralytics.com/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/ultralytics">GitHub</a> for support, and join our <a href="https://discord.gg/n6cFeSPZdD">Discord</a> community for questions and discussions!

</div>

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="zMJ1XlSC8CveRtknmRR0")
project = rf.workspace("wallpaper-zotss").project("alpaco4th")
dataset = project.version(1).download("folder")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.1 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=62169b4b51163b976257255f5ea258e90d9a133b5bfb4daf1faebc587b6cfabb
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to alpaco4th-1 in folder:: 100%|██████████| 9019/9019 [00:01<00:00, 6103.90it/s]


In [3]:
!pip install transformers
!pip install torch
!pip install torchvision
!pip install tqdm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 39.2 MB/s eta 0:00:00


In [10]:
import os
import torch
import timm
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# 데이터 전처리
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 학습 데이터와 검증 데이터를 로드하고 전처리
train_data = ImageFolder("/content/alpaco4th-1/train", transform=transform)
val_data = ImageFolder("/content/alpaco4th-1/valid", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4)

In [11]:

# 모델 로드
model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True, num_classes=18)

# GPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 손실 함수와 옵티마이저
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=3e-5, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


In [ ]:
from sklearn.metrics import f1_score

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    true_labels = []
    predicted_labels = []
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    return running_loss / len(dataloader), f1

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}")
    val_loss, val_f1 = validate_epoch(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Weighted F1 Score: {val_f1:.4f}")
    scheduler.step()

# 모델 저장
torch.save(model.state_dict(), "fine_tuned_swin.pth")

Epoch 1/10


100%|██████████| 196/196 [01:20<00:00,  2.43it/s]


Train Loss: 0.1006


100%|██████████| 56/56 [00:09<00:00,  5.79it/s]


Validation Loss: 0.1785
Validation Weighted F1 Score: 0.9461
Epoch 2/10


...

100%|██████████| 196/196 [01:25<00:00,  2.30it/s]


Train Loss: 0.0509


100%|██████████| 56/56 [00:09<00:00,  5.98it/s]


Validation Loss: 0.1636
Validation Weighted F1 Score: 0.9543
Epoch 4/10


 66%|██████▌   | 129/196 [00:56<00:28,  2.31it/s]

In [19]:
from PIL import Image
from torch.utils.data import Dataset
import os

class UnlabeledDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

In [20]:
# 테스트 데이터를 로드하고 전처리
test_data = UnlabeledDataset("/content/drive/MyDrive/testfile/test", transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)

# 테스트 데이터 예측 함수 정의
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in tqdm(dataloader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
    return predictions

# 테스트셋 예측
test_predictions = predict(model, test_loader, device)

100%|██████████| 25/25 [00:15<00:00,  1.60it/s]


In [21]:
import pandas as pd
submit = pd.read_csv('/content/drive/MyDrive/testfile/sample_submission.csv')

In [23]:
submit

,id,label
0,TEST_000,훼손
1,TEST_001,훼손
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,훼손
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,훼손
790,TEST_790,훼손


In [22]:
submit['label'] = test_predictions

In [23]:
submit.to_csv('/content/baseline_submit.csv', index=False)

In [24]:
import csv

# 딕셔너리 생성
label_dict = {
    0: '가구수정',
    1: '걸레받이수정',
    2: '곰팡이',
    3: '꼬임',
    4: '녹오염',
    5: '들뜸',
    6: '면불량',
    7: '몰딩수정',
    8: '반점',
    9: '석고수정',
    10: '오염',
    11: '오타공',
    12: '울음',
    13: '이음부불량',
    14: '창틀문틀수정',
    15: '터짐',
    16: '피스',
    17: '훼손'
}

# CSV 파일 열기
with open('/content/baseline_submit.csv', mode='r') as file:
    reader = csv.reader(file)
    submission = list(reader)

# 레이블 번호를 한글로 변경
for i in range(1, len(submission)):
    submission[i][1] = label_dict[int(submission[i][1])]

# 새로운 CSV 파일로 저장
with open('/content/zzin_submission.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(submission)